# Coupling coefficients for IFS-AMIP

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import intake

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean.cm as cmo
import seaborn as sns

from datetime import datetime

from dask.diagnostics import ProgressBar

In [2]:
cd

/home/b/b382473


In [3]:
import geostats as gs

In [4]:
import iris
import iris.analysis as ia

In [5]:
def ifs_to_latlon(ds):
    '''
    Uses the latitude-longitude information encoded in the regular grid IFS output to reconstructed the regular grid
    '''
    return ds.rename({'value':'latlon'}).set_index(latlon=("lat","lon")).unstack("latlon")
def ifs_fix_time_for_monthly_data(ds):
    '''
    Monthly mean data has the time axis encoded incorrectly.
    This function shifts it back by one day
    '''
    return ds.assign_coords(time=ds['time']- pd.Timedelta('1D'))

def get_area(da,mask=False):
    print('Computing grid-box area')
    import iris.analysis as ia
    if 'time' in da.dims:
        da = da.isel(time=0).drop('time')
    d = da.to_iris()
    d.coord('longitude').guess_bounds()
    d.coord('latitude').guess_bounds()

    area_weights = ia.cartography.area_weights(d)
    area = xr.ones_like(da) * area_weights
    if mask:
        area = area.where(~np.isnan(da))
    area = area.rename('area').load()
    area.attrs['long_name'] = 'grid_box_area'
    area.attrs['units'] = 'm^2'
    return area

def print_var(ds,filt=None):
    '''
    Print variables (varname,name) in Dataset. 
    If <filt> is provided, print only those where <filt> is present in the <name> attribute (ds[shortname].attrs['name'])

    Usage: print_var(ds,'wind')
    Output: printout of variables in ds whose name (long name in attributes, not short name to access) contains 'wind', e.g.
        10si  :   10 metre wind speed
        10u  :   10 metre U wind component
        10v  :   10 metre V wind component
    '''
    if filt:
        [print('%10s  :   %s' % (d,ds[d].attrs['name'])) for d in ds if filt.lower() in ds[d].attrs['name'].lower()]
    else:
        [print('%10s  :   %s' % (d,ds[d].attrs['name'])) for d in ds]

## Open and inspect the catalogue

In [18]:
cat = intake.open_catalog('https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml')\
                          ['dkrz']['disk']['model-output']['ifs-amip']

In [19]:
list(cat)

['amip-hist-obs.atmos.gr025',
 'amip-hist-obs-lr30.atmos.gr025',
 'amip-hist-obs-c-lr30-a-0.atmos.gr025',
 'amip-hist-obs-c-lr30-a-lr30.atmos.gr025',
 'amip-ng-obs.atmos.gr025',
 'amip-ng-obs-lr30.atmos.gr025',
 'amip-hist-esav3.atmos.gr025',
 'amip-hist-esav3-c-0-a-lr30.atmos.gr025']

In [8]:
run = 'amip-hist-obs'

ds_2d_6h_nat = cat[run + '.atmos.gr025']['2D_6h_native'].to_dask()
ds_2d_6h_025 = ifs_to_latlon(cat[run + '.atmos.gr025']['2D_6h_0.25deg'].to_dask())

## Compute and write out daily mean surface winds on native grid: necessary to compute surface wind divergence

In [127]:
for year in np.arange(2010,2021):
    for month in np.arange(1,13):
        print(year,month)
        ds_2d_6h_nat[['10u','10v']].sel(time='%.4i-%.2i' % (year,month)).resample(time='1D').mean('time').to_netcdf(
            '/work/bk1377/b382473/native/IFS/OSTIA/wind/wind_10u_10v_%.4i-%.2i.nc' % (year,month)
        )

2010 1
2010 2
2010 3
2010 4
2010 5
2010 6
2010 7
2010 8
2010 9
2010 10
2010 11
2010 12
2011 1
2011 2
2011 3
2011 4
2011 5
2011 6
2011 7
2011 8
2011 9
2011 10
2011 11
2011 12
2012 1
2012 2
2012 3
2012 4
2012 5
2012 6
2012 7
2012 8
2012 9
2012 10
2012 11
2012 12
2013 1
2013 2
2013 3
2013 4
2013 5
2013 6
2013 7
2013 8
2013 9
2013 10
2013 11
2013 12
2014 1
2014 2
2014 3
2014 4
2014 5
2014 6
2014 7
2014 8
2014 9
2014 10
2014 11
2014 12
2015 1
2015 2
2015 3
2015 4
2015 5
2015 6
2015 7
2015 8
2015 9
2015 10
2015 11
2015 12
2016 1
2016 2
2016 3
2016 4
2016 5
2016 6
2016 7
2016 8
2016 9
2016 10
2016 11
2016 12
2017 1
2017 2
2017 3
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2017 12
2018 1
2018 2
2018 3
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11
2018 12
2019 1
2019 2
2019 3
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
2019 10
2019 11
2019 12
2020 1
2020 2
2020 3
2020 4
2020 5
2020 6
2020 7
2020 8
2020 9
2020 10
2020 11
2020 12


## Load SST gradients

In [9]:
obs = intake.open_catalog('https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml')\
                          ['dkrz']['disk']['observations']

In [10]:
if run == 'amip-hist-obs':
    # SST gradients on raw dataset
    ostia_SSTgrad = obs['OSTIA']['OSTIA']['SSTgrad_0_0'].to_dask()

elif run == 'amip-hist-obs-lr30':
    # SST gradients, with climatology filtered with LR30
    ostia_SSTgrad = obs['OSTIA']['OSTIA']['SSTgrad_LR30_0'].to_dask()

elif run == 'amip-hist-obs-c-lr30-a-0':
    # SST gradients, with anomalies filtered with LR30
    ostia_SSTgrad = obs['OSTIA']['OSTIA']['SSTgrad_0_LR30'].to_dask()

elif run == 'amip-hist-obs-c-lr30-a-lr30':
    # SST gradients, with climatology and anomalies filtered with LR30
    ostia_SSTgrad = obs['OSTIA']['OSTIA']['SSTgrad_LR30_LR30'].to_dask()

In [11]:
ostia_SSTgrad = ostia_SSTgrad.rename({'values':'value'})

# downwind SST gradient (winds) vs wind divergence
1. Compute downwind & cross-wind SST gradients
2. Compute near-surface wind divergence on native grid
3. Remap onto regular 0.25deg grid
4. Perform 30-day running mean on 0.25deg grid files
5. Spatially high-pass 30-day running mean
6. Compute coupling coefficients

## 1. Compute downwind & cross-wind SST gradients

In [13]:
def get_downT_crossT(tauu,tauv,dTdx,dTdy):
    #Compute downwind and crosswind SST gradients
    mtau=np.sqrt(tauu*tauu + tauv*tauv)
    cosphi=tauu/mtau
    sinphi=tauv/mtau
    print('wspd=',np.shape(mtau))
    print('cosphi=',np.shape(cosphi))
    print('sinphi=',np.shape(sinphi))
    del(mtau)

    downT=(cosphi*dTdx.squeeze()+sinphi*dTdy.squeeze()).rename('downT')
    crossT=(sinphi*dTdx.squeeze()-cosphi*dTdy.squeeze()).rename('crossT')
    
    return downT, crossT

In [14]:
windgrads = xr.merge(
    # get_downT_crossT(ds_2d_6h_nat['ewss'],ds_2d_6h_nat['nsss'],ostia_SSTgrad['dTdx'],ostia_SSTgrad['dTdy'])
    get_downT_crossT(ds_2d_6h_nat['ewss'].sel(time='2010'),ds_2d_6h_nat['nsss'],ostia_SSTgrad['dTdx'],ostia_SSTgrad['dTdy'])
)

wspd= (365, 654400)
cosphi= (365, 654400)
sinphi= (365, 654400)


In [117]:
with ProgressBar():
    windgrads.load()

[########################################] | 100% Completed | 8.48 ss


In [34]:
for year in np.unique(ds_2d_6h_nat['time.year'].values):
    print(year)
    SSTgradsi = xr.merge(
        get_downT_crossT(ds_2d_6h_nat['ewss'].sel(time='%i' % year),ds_2d_6h_nat['nsss'].sel(time='%i' % year),ostia_SSTgrad['dTdx'].sel(time='%i' % year),ostia_SSTgrad['dTdy'].sel(time='%i' % year))
    )
    with ProgressBar():
        SSTgradsi.to_netcdf('/work/bk1377/b382473/native/IFS/OSTIA/SSTgrads_wind/SSTgradsi_%.4i.nc' % year)

2010
wspd= (365, 654400)
cosphi= (365, 654400)
sinphi= (365, 654400)
[########################################] | 100% Completed | 11.56 s
2011
wspd= (365, 654400)
cosphi= (365, 654400)
sinphi= (365, 654400)
[########################################] | 100% Completed | 10.69 ss
2012
wspd= (366, 654400)
cosphi= (366, 654400)
sinphi= (366, 654400)
[########################################] | 100% Completed | 13.03 ss
2013
wspd= (365, 654400)
cosphi= (365, 654400)
sinphi= (365, 654400)
[########################################] | 100% Completed | 12.67 ss
2014
wspd= (365, 654400)
cosphi= (365, 654400)
sinphi= (365, 654400)
[########################################] | 100% Completed | 16.34 ss
2015
wspd= (365, 654400)
cosphi= (365, 654400)
sinphi= (365, 654400)
[########################################] | 100% Completed | 17.80 ss
2016
wspd= (366, 654400)
cosphi= (366, 654400)
sinphi= (366, 654400)
[########################################] | 100% Completed | 14.78 s
2017
wspd= (365, 65440

## 2. Compute near-surface wind divergence on native grid

## 3. Remap onto regular 0.25deg grid

### wind divergence

### SST gradients

## 4. Perform 30-day running mean on 0.25deg grid files

### 4.2 Load 0.25 degree SST gradients

In [7]:
SSTgrads_025 = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/SSTgrads_wind/SSTgradsi_*.nc',concat_dim='time',combine='nested')

### 4.1 Load wind divergence

In [8]:
winddiv = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/winddiv/OSTIA_winddiv_??????_IFS25.nc',concat_dim='time',combine='nested')['winddiv']

### 4.2 Compute 30-day running mean of downwind SST gradients and surface wind divergence

In [9]:
winddiv_30d = winddiv.rolling(time=30,center=True).mean()

In [10]:
downT_30d = SSTgrads_025['downT'].rolling(time=30,center=True).mean()

In [11]:
for year in np.arange(2010,2020):
    print(year)
    winddiv_30di = winddiv_30d.sel(time=str(year))
    with ProgressBar():
        winddiv_30di.load()
    winddiv_30di.to_netcdf('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d/winddiv_30d_%.4i.nc' % year)

    downT_30di = downT_30d.sel(time=str(year))
    with ProgressBar():
        downT_30di.load()
    downT_30di.to_netcdf('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d/downT_30d_%.4i.nc' % year)

2010
[########################################] | 100% Completed | 11.15 ss
[########################################] | 100% Completed | 14.62 ss
2011
[########################################] | 100% Completed | 13.82 s
[########################################] | 100% Completed | 26.45 s
2012
[########################################] | 100% Completed | 12.75 ss
[########################################] | 100% Completed | 25.72 s
2013
[########################################] | 100% Completed | 16.86 s
[########################################] | 100% Completed | 24.70 s
2014
[########################################] | 100% Completed | 13.00 s
[########################################] | 100% Completed | 25.27 s
2015
[########################################] | 100% Completed | 12.28 ss
[########################################] | 100% Completed | 27.42 s
2016
[########################################] | 100% Completed | 14.23 ss
[########################################] | 100% 

## 5. Spatially low-pass 30-day running mean

Use (in EERIE_hackathon_2023/pre-joint-hackathon-2024/mesoscale-air-sea-coupling/IFS-FESOM):
- compute_filter_year_month.py
- compute_filter_year_month.job
- submit_filter_year_month.sh

to spatially low-pass filter SST, downwind SST gradient, surface wind speed and surface wind divergence
using gcm-filters (Gaussian filter, 30 x Rossby radius, 300-1500 km)

Output currently written to 
/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d_filtered/

## 6. Compute coupling coefficients: See "Coupling" section

# SST vs wind/stress magnitude

## 1. compute wind speed magnitude and compute daily mean (based on 6-hourly snapshots)
For SST, simply select the midnight snapshot: IFS-AMIP has to diurnal cycle of foundation SST

In [9]:
speed = ((ds_2d_6h_025[['10u','10v']]**2).to_array().sum('variable')**0.5).resample(time='1D').mean()

In [10]:
sst = ds_2d_6h_025['sst'].sel(time=(ds_2d_6h_025['time.hour']==0))

## 30-day running mean of SST and surface wind speed

In [11]:
speed_30d = speed.rolling(time=30,center=True).mean().rename('10si')
sst_30d = sst.rolling(time=30,center=True).mean()

In [12]:
sst_30d['lon'].attrs['long_name'] = 'longitude'
sst_30d['lon'].attrs['standard_name'] = 'longitude'
sst_30d['lon'].attrs['units'] = 'degrees_east'

In [13]:
for year in np.arange(2010,2021):
    print(year)
    speed_30di = speed_30d.sel(time=str(year))
    with ProgressBar():
        speed_30di.load()
    speed_30di.to_netcdf('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d/speed_30d_%.4i.nc' % year)
    sst_30di = sst_30d.sel(time=str(year))
    with ProgressBar():
        sst_30di.load()
    sst_30di.to_netcdf('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d/sst_30d_%.4i.nc' % year)

2010
[########################################] | 100% Completed | 61.66 s
[########################################] | 100% Completed | 16.72 ss
2011
[########################################] | 100% Completed | 66.22 s
[########################################] | 100% Completed | 18.43 ss
2012
[########################################] | 100% Completed | 67.46 s
[########################################] | 100% Completed | 18.04 ss
2013
[########################################] | 100% Completed | 67.61 s
[########################################] | 100% Completed | 18.53 ss
2014
[########################################] | 100% Completed | 66.98 s
[########################################] | 100% Completed | 16.62 ss
2015
[########################################] | 100% Completed | 72.42 s
[########################################] | 100% Completed | 20.36 ss
2016
[########################################] | 100% Completed | 67.24 s
[########################################] | 100%

## 5. Spatially low-pass 30-day running mean

Use (in EERIE_hackathon_2023/pre-joint-hackathon-2024/mesoscale-air-sea-coupling/IFS-FESOM):
- compute_filter_year_month.py
- compute_filter_year_month.job
- submit_filter_year_month.sh

to spatially low-pass filter SST, downwind SST gradient, surface wind speed and surface wind divergence
using gcm-filters (Gaussian filter, 30 x Rossby radius, 300-1500 km)

Output currently written to 
/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d_filtered/

## 6. Compute coupling coefficients: See "Coupling" section

# Coupling
Compute coupling coefficients using pre-computed 30-day running mean daily mean fields, raw, low-pass and high-pass filtered.

"Coupling coefficients" here only mean a simple Pearson correlation coefficient over the whole 10-year timeseries at daily resolution.

## SST and wind speed

### load 30-day running means

In [8]:
speed_30d = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d/speed_30d_????.nc')['10si'].chunk({'lat':721//4,'lon':1440//4})
sst_30d = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d/sst_30d_????.nc')['sst'].chunk({'lat':721//4,'lon':1440//4})

### load low-pass filteterd 30-day running means

In [20]:
speed_30d_lp = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d_filtered/speed_30d_????-??.nc')['10si'].chunk({'lat':721//4,'lon':1440//4})
sst_30d_lp = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d_filtered/sst_30d_????-??.nc')['sst'].chunk({'lat':721//4,'lon':1440//4})

### compute high-pass filteterd 30-day running means

In [10]:
speed_30d_hp = speed_30d - speed_30d_lp
sst_30d_hp = sst_30d - sst_30d_lp

In [11]:
corr_sst_speed = xr.corr(sst_30d_hp,speed_30d_hp,'time')
# corr_sst_speed = xr.corr(sst_30d_hp.sel(time=slice('2017','2019')),speed_30d_hp.sel(time=slice('2017','2019')),'time')

In [12]:
with ProgressBar():
    corr_sst_speed.load()

[################################        ] | 82% Completed | 285.41 s

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/array/numpy_compat.py:51: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 299.06 s


In [13]:
corr_sst_speed.rename('corr_sst_speed').to_netcdf('corr_sst_speed.nc')

In [9]:
corr_sst_speed_full = xr.corr(sst_30d,speed_30d,'time')
# corr_sst_speed_full = xr.corr(sst_30d.sel(time=slice('2017','2019')),speed_30d.sel(time=slice('2017','2019')),'time')

In [10]:
with ProgressBar():
    corr_sst_speed_full.load()

[##############################          ] | 76% Completed | 95.36 ss

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/array/numpy_compat.py:51: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


[######################################  ] | 96% Completed | 116.10 s

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 117.41 s


In [11]:
corr_sst_speed_full.rename('corr_sst_speed_full').to_netcdf('corr_sst_speed_full.nc')

In [21]:
corr_sst_speed_lp = xr.corr(sst_30d_lp,speed_30d_lp,'time')
# corr_sst_speed_lp = xr.corr(sst_30d_lp.sel(time=slice('2017','2019')),speed_30d_lp.sel(time=slice('2017','2019')),'time')

In [22]:
with ProgressBar():
    corr_sst_speed_lp.load()

[#############################           ] | 74% Completed | 85.70 ss

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/array/numpy_compat.py:51: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 95.42 s


In [24]:
corr_sst_speed_lp.rename('corr_sst_speed_lp').to_netcdf('corr_sst_speed_lp.nc')

## downwind SST gradient and wind divergence

### load 30-day running means

In [6]:
downT_30d = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d/downT_30d_????.nc')['downT'].chunk({'lat':721//4,'lon':1440//4}).assign_coords(time=pd.date_range('2010-01-01','2019-12-31'))
winddiv_30d = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d/winddiv_30d_????.nc')['winddiv'].chunk({'lat':721//4,'lon':1440//4})

### load low-pass filteterd 30-day running means

In [12]:
downT_30d_lp = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d_filtered/downT_30d_????-??.nc')['downT'].chunk({'lat':721//4,'lon':1440//4}).assign_coords(time=pd.date_range('2010-01-01','2019-12-31'))
winddiv_30d_lp = xr.open_mfdataset('/work/bk1377/b382473/reg25/ifsamip/OSTIA/mean_30d_filtered/winddiv_30d_????-??.nc')['winddiv'].chunk({'lat':721//4,'lon':1440//4})

### compute high-pass filteterd 30-day running means

In [16]:
downT_30d_hp = downT_30d - downT_30d_lp
winddiv_30d_hp = winddiv_30d - winddiv_30d_lp

In [17]:
corr_downT_winddiv = xr.corr(downT_30d_hp,winddiv_30d_hp,'time')
# corr_downT_winddiv = xr.corr(downT_30d_hp.sel(time=slice('2017','2019')),winddiv_30d_hp.sel(time=slice('2017','2019')),'time')

In [18]:
with ProgressBar():
    corr_downT_winddiv.load()

[################################        ] | 81% Completed | 171.79 s

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/array/numpy_compat.py:51: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


[###################################     ] | 89% Completed | 176.83 s

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 182.01 s


In [19]:
corr_downT_winddiv.rename('corr_downT_winddiv').to_netcdf('corr_downT_winddiv.nc')

In [7]:
corr_downT_winddiv_full = xr.corr(downT_30d,winddiv_30d,'time')
# corr_downT_winddiv_full = xr.corr(downT_30d.sel(time=slice('2017','2019')),winddiv_30d.sel(time=slice('2017','2019')),'time')

In [8]:
with ProgressBar():
    corr_downT_winddiv_full.load()

[###########################             ] | 68% Completed | 104.81 s

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/array/numpy_compat.py:51: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


[############################            ] | 71% Completed | 105.15 s

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 130.12 s


In [9]:
corr_downT_winddiv_full.rename('corr_downT_winddiv_full').to_netcdf('corr_downT_winddiv_full.nc')

In [13]:
corr_downT_winddiv_lp = xr.corr(downT_30d_lp,winddiv_30d_lp,'time')
# corr_downT_winddiv_lp = xr.corr(downT_30d_lp.sel(time=slice('2017','2019')),winddiv_30d_lp.sel(time=slice('2017','2019')),'time')

In [14]:
with ProgressBar():
    corr_downT_winddiv_lp.load()

[#############################           ] | 74% Completed | 81.02 ss

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/array/numpy_compat.py:51: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


[###############################         ] | 79% Completed | 82.71 s

/work/bk1377/b382473/conda/envs/science39/lib/python3.9/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 89.39 s


In [15]:
corr_downT_winddiv_lp.rename('corr_downT_winddiv_lp').to_netcdf('corr_downT_winddiv_lp.nc')